## PURPOSE OF THE NOTEBOOK + TITLE OF THE PROJECT

In [22]:
import requests
from requests.adapters import HTTPAdapter
import json

import pandas as pd
from datetime import date
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
import time
import numpy as np
import re

In [44]:
links = ['https://www.ravelry.com/patterns/search#pc=medical&sort=name&view=captioned_thumbs&page=1','https://www.ravelry.com/patterns/search#pc=medical&sort=name&view=captioned_thumbs&page=2','https://www.ravelry.com/patterns/search#pc=medical&sort=name&view=captioned_thumbs&page=3','https://www.ravelry.com/patterns/search#pc=medical&sort=name&view=captioned_thumbs&page=4']

### Get the ID's of all the recipes in the medical knitting recipe category

In [45]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

url = links[0]
await page.goto(url)

# Use playwright to login to the site. The xxxxx replace the username and password:
await page.locator("#user_login").fill('xxxxx')
await page.locator("#user_password").fill('xxxxx')
await page.locator("#login_form > div.splash_page__submit > div.splash_page__submit__login > button").click()

time.sleep(5)

pattern_ids = []

# Scrape each of the four pages:
for link in links:
    await page.goto(link)
    time.sleep(5)
    html = await page.content()
    doc = BeautifulSoup(html)

    content_card = doc.find_all(class_= "pattern captioned_pattern captioned_search_result")

    pattern_id = []

    # Loop through each pattern on the page and get the information:
    for card in content_card:
        pattern = {}
        pattern['name'] = card.find('a').get('title')
        pattern['url'] = card.find('a').get('href')

        find_id = card.get('id')
        pattern['id'] = re.findall(r'\d+', find_id)[0]

        # And append the pattern data to the list:
        pattern_id.append(pattern)
    pattern_ids.append(pattern_id)

### And take the list of lists (with some dictionaries inside) created in the scrape and turn it into one flat list.

This allows me to turn it into a dataframe and then into a csv file.

In [ ]:
flat_list = []
for ids in pattern_ids:
    for id in ids:
        flat_list.append(id)

id_df = pd.DataFrame(flat_list)

In [ ]:
id_df.to_csv("data/patterns.csv", index=False)

## Get information on the patterns

In [2]:
df = pd.read_csv("data/patterns.csv")

In [3]:
url_list = df.url.values.tolist()

In [ ]:
project_numbers = []

for url in url_list:
    await page.goto(url)

    time.sleep(0.5)

    project = {}

    html = await page.content()
    doc = BeautifulSoup(html)

    type = doc.find_all(class_='value')
    type = type[1].text
    type = re.sub(r'\n', '', type)

    project['craft_type'] = type
    
    projects = doc.find(class_= "pattern_people summary_box_people").text
    # replace \n\n\n with nothing
    projects = re.sub(r'\n\n\n', '', projects)
    project['project_numbers'] = re.sub(r' projec\D*,\nin \d* queues\n\n', '', projects)
    
    project_numbers.append(project)
await browser.close()

In [ ]:
df = pd.DataFrame(project_numbers)
df.to_csv("data/patterns_info.csv", index=False)

## WHAT IS HAPPENING HERE

In [4]:
patterns = pd.read_csv("data/patterns.csv")
patterns_info = pd.read_csv("data/patterns_info.csv")

In [5]:
# Merge the two dataframes using concat
total = pd.concat([patterns, patterns_info], axis=1)

# Give the recipe with no projects the value 0
total = total.replace(regex=r'no projects\n', value=0)

In [6]:
# And save it as a csv
total.to_csv("data/patterns_total.csv", index=False)

For the next part I have to manually categorize the recipes.

In [13]:
df = pd.read_csv("data/patterns_total_categorized.csv")

df['knocker'] = np.where(df['Category'] == 'knockers', 1, 0)
df

,name,url,id,craft_type,project_numbers,Category,knocker
0,"""Hey, at Least It Wasn't Your Leg"" Arm Sling b...",https://www.ravelry.com/patterns/library/hey-a...,80882,Knitting,5,injury,0
1,"""Kitty"" Eye Patch by Christine H. Wilkins",https://www.ravelry.com/patterns/library/kitty...,118789,Knitting,5,injury,0
2,Alternative ML Knitted Knockers by Makoshark,https://www.ravelry.com/patterns/library/alter...,951015,Knitting,4,knockers,1
3,Amon ammeinnlegg by Anne Tvedt,https://www.ravelry.com/patterns/library/amon-...,807316,Knitting,0,maternity,0
4,Amputee's Stump Covers by Suzetta Williams,https://www.ravelry.com/patterns/library/amput...,287776,Crochet,12,amputee,0
...,...,...,...,...,...,...,...
165,Woolly Boob by Kathryn Cottam,https://www.ravelry.com/patterns/library/wooll...,639320,Crochet,1,maternity,0
166,Worry Bear by Joanne Seiff,https://www.ravelry.com/patterns/library/worry...,908676,Knitting,3,mental_psycological,0
167,Wrapped in Lace PICC Line Cover by Kirsten Hol...,https://www.ravelry.com/patterns/library/wrapp...,749754,Crochet,9,cannula_iv,0
168,Wrist rest by Lynne Rowe,https://www.ravelry.com/patterns/library/wrist...,464528,Knitting,3,pain_relief,0


In [8]:
knockers_true_false = df.groupby('knocker').agg({'project_numbers':'sum'}).reset_index().sort_values(by='project_numbers', ascending=False)
knockers_true_false

,knocker,project_numbers
1,1,4744
0,0,991


In [17]:
# Get a df only with the knockers recipes. I will have to manually gather the price based on the yarn, but there aren't that many, so it is ok.
# Only keep the rows where knocker is 1
df1 = df.loc[df['knocker'] == 1]

df1.to_csv('data/knockers_recipes.csv', index=False)

What is the most popular category?

In [ ]:
categories = df.groupby('Category').agg({'project_numbers':'sum'}).reset_index().sort_values(by='project_numbers', ascending=False)
categories.drop(categories[categories['Category'] == 'na'].index, inplace = True)

categories = categories.head(5)
categories

,Category,project_numbers,knocker
6,knockers,4744,1
4,heating_cooling,261,0
5,injury,252,0
8,menstruation,196,0
1,cannula_iv,58,0


In [ ]:
categories.to_csv("data/categories.csv", index=False)

Are knockers most often knitted or croched?

In [ ]:
knockers = df[df['Category'] == 'knockers']
knockers_grouped = knockers.groupby('craft_type').agg({'project_numbers':'sum'}).reset_index().sort_values(by='project_numbers', ascending=False)

In [ ]:
knockers_grouped['percent'] = knockers_grouped.project_numbers / knockers_grouped.project_numbers.sum() * 100
knockers_grouped = knockers_grouped.round(0)
knockers_grouped

,craft_type,project_numbers,percent
1,Knitting,4441,94.0
0,Crochet,301,6.0
2,Loom Knitting,2,0.0


In [ ]:
knockers_grouped.to_csv("data/knockers_grouped.csv", index=False)

### Get the estimated price for a knocker

Measured by the price of the yarn needed to complete the project.

In [53]:
yarn_df = pd.read_csv("data/knockers_yarn_details.csv")
yarn_df = yarn_df.replace('na',np.NaN)

# Drop the recipes where some of the values are missing - they cannot be used in this calculation.
yarn_df = yarn_df.dropna()

#Figure out if you need one or two yarn wrenches
# First change the numeric columns into the right format
yarn_df['recipe_yardage_min'] = yarn_df['recipe_yardage_min'].astype(int)
yarn_df['recipe_yardage_max'] = yarn_df['recipe_yardage_max'].astype(int)
yarn_df['price_usd'] = yarn_df['price_usd'].astype(float)
yarn_df['yarn_yards'] = yarn_df['yarn_yards'].astype(int)
yarn_df['yard_grams'] = yarn_df['yard_grams'].astype(int)


# If the result is larger than 1, you will need more than one yarn wrench for the average version of the recipe. 
yarn_df['wrench'] = (yarn_df['recipe_yardage_min'] / yarn_df['recipe_yardage_max'] * 2) / yarn_df['yarn_yards'] * 100

yarn_df.loc[yarn_df['wrench'] > 1, 'wrench'] = 2
yarn_df.loc[yarn_df['wrench'] <= 1, 'wrench'] = 1

# And finally calculate the price of a knitted knocker
yarn_df['knocker_price'] = yarn_df['price_usd'] * yarn_df['wrench']
yarn_df.head(5)

,name,url,id,craft_type,project_numbers,Category,knocker,recipe_yardage_min,recipe_yardage_max,yarn,yarn_producer,price_usd,material,yarn_yards,yard_grams,wrench,knocker_price
2,Breast Cancer Awareness Boobies by Melanie Gro...,https://www.ravelry.com/patterns/library/breas...,865422,Crochet,4,knockers,1,180,191,100% cotton double knit,MoYa,8.88,cotton,148,50,2.0,17.76
4,Breast Forms by Lisa Rode,https://www.ravelry.com/patterns/library/breas...,865397,Crochet,0,knockers,1,40,140,Simply Soft Solids,Caron,5.49,acrylic,315,170,1.0,5.49
5,Breast Prosthetic Round by Scarlett Royal,https://www.ravelry.com/patterns/library/breas...,1053583,Loom Knitting,0,knockers,1,90,110,Coboo,Lion Brand,5.99,cotton_bamboo,232,100,1.0,5.99
6,Bust Buddies by Eileen Adler,https://www.ravelry.com/patterns/library/bust-...,825548,Knitting,0,knockers,1,50,50,Ultra Pima,Cascade Yarns,13.50,pima,220,100,1.0,13.50
8,Crochet Breast by Denise Hayes,https://www.ravelry.com/patterns/library/croch...,663244,Crochet,0,knockers,1,137,164,Magic Light,Ice Yarns,7.99,acrylic,394,100,1.0,7.99


Create a new dataframe containing name and price of both the knitted knockers and conventional breast prosteses. 

First I reduce the df above, then I read in another dataset (the conventional breast prosteses) and then I merge the two dataframes

In [63]:
knocker = yarn_df[['name','Category','knocker_price']]

# Rename the columns to match the other dataframe
knocker = knocker.rename(columns={'Category':'type','knocker_price':'price'})

In [61]:
# Load in the conventional prosthesis data
prosthesis = pd.read_csv("data/prosthesis_info.csv")
prosthesis = prosthesis.drop('retailer', axis=1)

In [64]:
# Concatenate the two dataframes
comparison = pd.concat([knocker, prosthesis], ignore_index=True, axis=0)
comparison

,name,type,price
0,Breast Cancer Awareness Boobies by Melanie Gro...,knockers,17.76
1,Breast Forms by Lisa Rode,knockers,5.49
2,Breast Prosthetic Round by Scarlett Royal,knockers,5.99
3,Bust Buddies by Eileen Adler,knockers,13.50
4,Crochet Breast by Denise Hayes,knockers,7.99
5,Gebreide Borstprothese by Claudia Barbo,knockers,4.50
6,Knitted Knockers - Flat On Two Needles by Terr...,knockers,13.50
7,Knitted Knockers Double pointed needles by Kni...,knockers,13.50
8,Knitted Knockers DPNs In Round by Terry Neal,knockers,13.50
9,Knitted Knockers on DPNs by Knitted Knockers,knockers,7.95


Use the Ravelry API to get information about projects that uses each of the recipes in the medical knitting recipe category.

In [ ]:
pattern_info = []
for id in col_list:
    api_key_id = 'xxxxxx'
    api_key_secret = 'xxxxxx'
    headers = {'Content-Type': 'application/json'}
    auth = (api_key_id, api_key_secret)
    newSession = requests.Session()
    link = f"https://api.ravelry.com/patterns/{id}/projects.json"
    response = newSession.post('https://api.ravelry.com/patterns/80882/projects.json',auth=auth,headers=headers,verify=False) 
    print(response.status_code)
    data = response.json()
    pattern_info.append(data)

In [ ]:
api_key_id = 'xxxxxx'
api_key_secret = 'xxxxxx'
headers = {'Content-Type': 'application/json'}
auth = (api_key_id, api_key_secret)
newSession = requests.Session()
response = newSession.post('https://api.ravelry.com/patterns/search.json?category=medical',auth=auth,headers=headers,verify=False) 
print(response.status_code)

In [ ]:
data = response.json()

In [ ]:

api_key_id = 'xxxxxx'
api_key_secret = 'xxxxxx'
headers = {'Content-Type': 'application/json'}
auth = (api_key_id, api_key_secret)
newSession = requests.Session()
response = newSession.post('https://api.ravelry.com/patterns/503114/projects.json?page_size=5000',auth=auth,headers=headers,verify=False) 
print(response.status_code)
pattern_data = response.json()
len(pattern_data['projects'])
